#### 데이터 전처리 최종마무리

In [1]:
# 필요 라이브러리 등록
import pandas as pd

In [2]:
# 서울시 스타벅스 매장 목록 불러오기
dfStarbucksSeoul = pd.read_excel('./data/starbucks_seoul_list.xlsx', usecols="B:F")
dfStarbucksSeoul.tail()

,매장명,위도,경도,매장타입,주소
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310
609,상봉역,37.596890,127.086470,general,서울특별시 중랑구 망우로 307 (상봉동)
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층"
611,양원역,37.606654,127.106360,general,서울특별시 중랑구 양원역로10길 3 (망우동)
612,중화역,37.601709,127.078411,general,서울특별시 중랑구 봉화산로 35 1층


In [3]:
# 주소를 ' '(공백)으로 자른다음 배열의 1번째 인덱스값이 구가 된다.
guName = []
for address in dfStarbucksSeoul['주소']:
    guName.append(address.split(' ')[1])

dfStarbucksSeoul['구'] = guName

In [4]:
dfStarbucksSeoul.tail()

,매장명,위도,경도,매장타입,주소,구
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,중랑구
609,상봉역,37.596890,127.086470,general,서울특별시 중랑구 망우로 307 (상봉동),중랑구
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",중랑구
611,양원역,37.606654,127.106360,general,서울특별시 중랑구 양원역로10길 3 (망우동),중랑구
612,중화역,37.601709,127.078411,general,서울특별시 중랑구 봉화산로 35 1층,중랑구


In [5]:
dfStarbucksSeoul.to_excel('./data/starbucks_seoul_list_final.xlsx', index=False)

In [6]:
# 스타벅스 분석 데이터 만들기
dfSeoulGu = pd.read_excel('./data/서울시_위경도.xlsx')
dfSeoulGu.tail()

,docity,do,city,longitude,latitude
20,서울용산구,서울,용산구,126.967522,37.536094
21,서울은평구,서울,은평구,126.931242,37.599969
22,서울종로구,서울,종로구,126.981642,37.570378
23,서울중구,서울,중구,126.999642,37.561003
24,서울중랑구,서울,중랑구,127.094778,37.603806


In [7]:
# docity 컬럼 삭제, 위경도 위치 변경
dfSeoulGuLast = dfSeoulGu[['do', 'city', 'latitude', 'longitude']]

In [8]:
# 각 컬럼 이름을 한글로 변경
dfSeoulGuLast.columns = ['시', '구', '위도', '경도']

In [9]:
dfSeoulGuLast.tail()

,시,구,위도,경도
20,서울,용산구,37.536094,126.967522
21,서울,은평구,37.599969,126.931242
22,서울,종로구,37.570378,126.981642
23,서울,중구,37.561003,126.999642
24,서울,중랑구,37.603806,127.094778


In [10]:
dfSeoulGuLast.to_excel('./data/최종_서울_구리스트.xlsx', index=False)

In [11]:
# 스타벅스 매장에서 구 를 기준으로 그룹화, 매장명 수 카운팅
starbucksSeoulCount = dfStarbucksSeoul.pivot_table(values='매장명', index='구', aggfunc='count').rename(
    columns={'매장명': '매장수'})
starbucksSeoulCount.tail()

,매장수
구,
용산구,24
은평구,14
종로구,40
중구,54
중랑구,8


In [12]:
# 서울시_구리스트에 스타벅스 구별 매장수를 병합(merge)
lastSbSeoulCount = pd.merge(left=dfSeoulGuLast, right=starbucksSeoulCount, how='left', on='구')

In [13]:
# 서울시 구별 인구통계 로드
dfSeoulGuPop = pd.read_excel('./data/서울_구별인구_수정.xlsx')
dfSeoulGuPop.tail()

,구,세대소계,성별소계,남자,여자,한국인소계,남자.1,여자.1,등록외국인소계,남자.2,여자.2,세대당인구,65세이상인구
21,관악구,284578,497883,249026,248857,481956,242651,239305,15927,6375,9552,1.69,85418
22,서초구,169884,412078,196391,215687,407664,194291,213373,4414,2100,2314,2.40,65661
23,강남구,239775,550282,262991,287291,544873,260520,284353,5409,2471,2938,2.27,86606
24,송파구,285927,660025,316981,343044,654166,314347,339819,5859,2634,3225,2.29,108894
25,강동구,203734,463318,226237,237081,459167,224423,234744,4151,1814,2337,2.25,82984


In [14]:
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount, right=dfSeoulGuPop[['구', '성별소계']], how='left', on='구')

In [15]:
lastSbSeoulCount

,시,구,위도,경도,매장수,성별소계
0,서울,강남구,37.514575,127.049556,91,550282
1,서울,강동구,37.527367,127.125864,17,463318
2,서울,강북구,37.636956,127.027719,6,292977
3,서울,강서구,37.548156,126.851675,27,568826
4,서울,관악구,37.475386,126.953844,12,497883
5,서울,광진구,37.535739,127.084533,18,351180
6,서울,구로구,37.492650,126.889597,14,415651
7,서울,금천구,37.449108,126.904197,13,241105
8,서울,노원구,37.651461,127.058389,14,502925
9,서울,도봉구,37.665833,127.049522,7,309494


In [16]:
lastSbSeoulCount.columns = ['시', '구', '위도', '경도', '스타벅스매장수', '주민등록인구']

In [17]:
# 사업체 정보 로드
dfSeoulGuBiz = pd.read_excel('./data/서울_구별사업체현황_수정.xlsx')
dfSeoulGuBiz.tail()

,구,소계,여성대표자,성별소계,남자,여자
21,관악구,37915,13641,128417,68392,60025
22,서초구,71958,23066,487976,290084,197892
23,강남구,110007,35453,801419,447809,353610
24,송파구,74531,25400,400781,244588,156193
25,강동구,41660,15226,157465,83415,74050


In [18]:
# 사업체정보에서 컬럼이름 부터 변경
dfSeoulGuBiz = dfSeoulGuBiz[['구', '소계', '성별소계']]
dfSeoulGuBiz.columns = ['구', '사업체수', '종사자수']

In [19]:
# 사업체정보를 스타벅스 최종데이터에 병합
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount, right=dfSeoulGuBiz, how='left', on='구')

In [20]:
lastSbSeoulCount.to_excel('./data/최종_서울_스타벅스매장_데이터.xlsx')

#### 데이터 시각화

In [21]:
# 시각화 라이브러리 등록
import folium
import json

In [22]:
# 엑셀에서 불러옴
dfStarbucksCountSeoul = pd.read_excel('./data/최종_서울_스타벅스매장_데이터.xlsx')
dfStarbucksCountSeoul.tail()

,Unnamed: 0,시,구,위도,경도,스타벅스매장수,주민등록인구,사업체수,종사자수
20,20,서울,용산구,37.536094,126.967522,24,227106,29680,152605
21,21,서울,은평구,37.599969,126.931242,14,470869,36509,109031
22,22,서울,종로구,37.570378,126.981642,40,150453,48361,275063
23,23,서울,중구,37.561003,126.999642,54,131793,70308,386564
24,24,서울,중랑구,37.603806,127.094778,8,387470,39310,113144


In [23]:
# 매장 타입 개수
dfStarbucksSeoul['매장타입'].value_counts()

매장타입
general      552
reserve       36
generalDT     23
generalWT      2
Name: count, dtype: int64

In [24]:
seoulLoc = [37.550800, 126.989857]
sbMap = folium.Map(location=seoulLoc, zoom_start=11)

for idx in dfStarbucksSeoul.index:
    lat = dfStarbucksSeoul.loc[idx, '위도']
    lng = dfStarbucksSeoul.loc[idx, '경도']
    storeType = dfStarbucksSeoul.loc[idx, '매장타입']
    storeName = dfStarbucksSeoul.loc[idx, '매장명']

    # 매장 타입별 색상
    fillColor = ''
    if storeType == 'general':
        fillColor = 'darkgreen'
        size = 2
    elif storeType == 'reserve':
        fillColor = 'black'
        size = 5
    elif storeType == 'generalDT':
        fillColor = 'orange'
        size = 3
    elif storeType == 'generalWT':
        fillColor = 'purple'
        size = 5

    # 서클마커
    folium.CircleMarker(location=[lat, lng], fill=True, fill_color=fillColor,
                        fill_opacity=1, color='yellow', weight=1,
                        popup=storeName, radius=size).add_to(sbMap)

sbMap

In [25]:
# 서울시 구별 스타벅스 매장수 시각화
dfStarbucksSeoulCount = pd.read_excel('./data/최종_서울_스타벅스매장_데이터.xlsx', usecols='B:I')
dfStarbucksSeoulCount.tail()

,시,구,위도,경도,스타벅스매장수,주민등록인구,사업체수,종사자수
20,서울,용산구,37.536094,126.967522,24,227106,29680,152605
21,서울,은평구,37.599969,126.931242,14,470869,36509,109031
22,서울,종로구,37.570378,126.981642,40,150453,48361,275063
23,서울,중구,37.561003,126.999642,54,131793,70308,386564
24,서울,중랑구,37.603806,127.094778,8,387470,39310,113144


In [26]:
path = './data/seoul_geo.json'
seoulGeoSimple = json.load(open(path, encoding='utf-8'))

In [27]:
seoulGeoSimple['features'][0]['properties']

{'code': '11250',
 'name': '강동구',
 'name_eng': 'Gangdong-gu',
 'base_year': '2013'}

In [28]:
def style_function(feature):
    return {
        'opacity': 0.9,
        'weight': 4,
        'color': 'white',
        'fillOpacity': 0,
        'dashArray': '5, 5'
    }

In [29]:
# 지도생성
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11, tiles='CartoDB dark_matter')

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)
sbCountMap

In [30]:
# 스타벅스 평균 매장수 계산
sbMean = dfStarbucksSeoulCount['스타벅스매장수'].mean()

In [31]:
# 지도생성
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11, tiles='CartoDB dark_matter')

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx, '위도']
    lng = dfStarbucksSeoulCount.loc[idx, '경도']
    count = dfStarbucksSeoulCount.loc[idx, '스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx, '구']

    if count > sbMean:
        fillColor = 'red'  # 평균보다 크면 빨강
    else:
        fillColor = 'blue'

    folium.CircleMarker(location=[lat, lng], color='#FFFF00', fill_color=fillColor, fill_opacity=0.7,
                        popup=guName + '/' + str(count),
                        weight=1.5, radius=count / 2).add_to(sbCountMap)

sbCountMap

In [32]:
from folium.features import DivIcon

In [33]:
dfStarbucksSeoulCount.loc[8, '위도'] = 37.653320
dfStarbucksSeoulCount.loc[8, '경도'] = 127.074440

dfStarbucksSeoulCount.loc[23, '위도'] = 37.561141
dfStarbucksSeoulCount.loc[23, '경도'] = 126.993101
dfStarbucksSeoulCount

,시,구,위도,경도,스타벅스매장수,주민등록인구,사업체수,종사자수
0,서울,강남구,37.514575,127.049556,91,550282,110007,801419
1,서울,강동구,37.527367,127.125864,17,463318,41660,157465
2,서울,강북구,37.636956,127.027719,6,292977,26385,80222
3,서울,강서구,37.548156,126.851675,27,568826,58806,290473
4,서울,관악구,37.475386,126.953844,12,497883,37915,128417
5,서울,광진구,37.535739,127.084533,18,351180,33706,129707
6,서울,구로구,37.492650,126.889597,14,415651,52791,240689
7,서울,금천구,37.449108,126.904197,13,241105,47964,255449
8,서울,노원구,37.653320,127.074440,14,502925,37909,133398
9,서울,도봉구,37.665833,127.049522,7,309494,25391,79097


In [34]:
# 서울시 구별 스타벅스 매장수 시각화
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11)

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple, data=dfStarbucksSeoulCount, columns=['구', '스타벅스매장수'],
                  legend_name='구 별 스타벅스매장수',
                  fill_color='YlGn', fill_opacity=0.7, line_opacity=0.5, key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx, '위도']
    lng = dfStarbucksSeoulCount.loc[idx, '경도']
    count = dfStarbucksSeoulCount.loc[idx, '스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx, '구']

    folium.Marker(location=[lat, lng], icon=DivIcon(
        icon_size=(150, 60),
        icon_anchor=(75, 25),
        html=f'''
        <div style="font-size: 9pt; font-weight: bold; color: black;text-align: center;">
            {guName} / {count}
        </div>
        '''
    )).add_to(sbCountMap)

sbCountMap

In [35]:
# 서울시 구별 인구분포 시각화
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11)

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple, data=dfStarbucksSeoulCount, columns=['구', '주민등록인구'],
                  legend_name='구 별 인구분포',
                  fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.5, key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx, '위도']
    lng = dfStarbucksSeoulCount.loc[idx, '경도']
    count = dfStarbucksSeoulCount.loc[idx, '주민등록인구']
    guName = dfStarbucksSeoulCount.loc[idx, '구']

    folium.Marker(location=[lat, lng], icon=DivIcon(
        icon_size=(150, 60),
        icon_anchor=(75, 25),
        html=f'''
        <div style="font-size: 9pt; font-weight: bold; color: black;text-align: center;">
            {guName} / {count}
        </div>
        '''
    )).add_to(sbCountMap)

sbCountMap

In [36]:
# 서울시 구별 사업체수 시각화
sbCountMap = folium.Map(location=seoulLoc, zoom_start=11)

folium.GeoJson(seoulGeoSimple, style_function=style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple, data=dfStarbucksSeoulCount, columns=['구', '사업체수'],
                  legend_name='구 별 사업체분포',
                  fill_color='PuBu', fill_opacity=0.7, line_opacity=0.5, key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx, '위도']
    lng = dfStarbucksSeoulCount.loc[idx, '경도']
    count = dfStarbucksSeoulCount.loc[idx, '주민등록인구']
    guName = dfStarbucksSeoulCount.loc[idx, '구']

    folium.Marker(location=[lat, lng], icon=DivIcon(
        icon_size=(150, 60),
        icon_anchor=(75, 25),
        html=f'''
        <div style="font-size: 9pt; font-weight: bold; color: black;text-align: center;">
            {guName} / {count}
        </div>
        '''
    )).add_to(sbCountMap)

sbCountMap

#### 결론
- 스타벅스는 인구수와 별개로 사업체가 많아서 직장인이 분포하는 구에 매장을 많이 오픈한다.